# Tutorial Part 12: Using Reinforcement Learning to Play Pong

This notebook demonstrates using reinforcement learning to train an agent to play Pong.

The first step is to create an `Environment` that implements this task.  Fortunately,
OpenAI Gym already provides an implementation of Pong (and many other tasks appropriate
for reinforcement learning).  DeepChem's `GymEnvironment` class provides an easy way to
use environments from OpenAI Gym.  We could just use it directly, but in this case we
subclass it and preprocess the screen image a little bit to make learning easier.

To install `gym` you should use `pip install 'gym[atari]'` (We need the extra modifier since we'll be using an atari game). We'll add this command onto our usual Colab installation commands for you

In [ ]:
!wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
!chmod +x Anaconda3-2019.10-Linux-x86_64.sh
!bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc
!conda install pip
!pip install 'gym[atari]'

In [1]:
import deepchem as dc
import numpy as np

class PongEnv(dc.rl.GymEnvironment):
  def __init__(self):
    super(PongEnv, self).__init__('Pong-v0')
    self._state_shape = (80, 80)
  
  @property
  def state(self):
    # Crop everything outside the play area, reduce the image size,
    # and convert it to black and white.
    cropped = np.array(self._state)[34:194, :, :]
    reduced = cropped[0:-1:2, 0:-1:2]
    grayscale = np.sum(reduced, axis=2)
    bw = np.zeros(grayscale.shape)
    bw[grayscale != 233] = 1
    return bw

  def __deepcopy__(self, memo):
    return PongEnv()

env = PongEnv()

/Users/bharath/opt/anaconda3/envs/deepchem/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
RDKit WARNING: [17:47:46] Enabling RDKit 2019.09.3 jupyter extensions
/Users/bharath/opt/anaconda3/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bharath/opt/anaconda3/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Pas

Next we create a network to implement the policy.  We begin with two convolutional layers to process
the image.  That is followed by a dense (fully connected) layer to provide plenty of capacity for game
logic.  We also add a small Gated Recurrent Unit.  That gives the network a little bit of memory, so
it can keep track of which way the ball is moving.

We concatenate the dense and GRU outputs together, and use them as inputs to two final layers that serve as the
network's outputs.  One computes the action probabilities, and the other computes an estimate of the
state value function.

We also provide an input for the initial state of the GRU, and returned its final state at the end.  This is required by the learning algorithm

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Conv2D, Dense, Flatten, GRU, Reshape

class PongPolicy(dc.rl.Policy):
    def __init__(self):
        super(PongPolicy, self).__init__(['action_prob', 'value', 'rnn_state'], [np.zeros(16)])

    def create_model(self, **kwargs):
        state = Input(shape=(80, 80))
        rnn_state = Input(shape=(16,))
        conv1 = Conv2D(16, kernel_size=8, strides=4, activation=tf.nn.relu)(Reshape((80, 80, 1))(state))
        conv2 = Conv2D(32, kernel_size=4, strides=2, activation=tf.nn.relu)(conv1)
        dense = Dense(256, activation=tf.nn.relu)(Flatten()(conv2))
        gru, rnn_final_state = GRU(16, return_state=True, return_sequences=True)(
            Reshape((-1, 256))(dense), initial_state=rnn_state)
        concat = Concatenate()([dense, Reshape((16,))(gru)])
        action_prob = Dense(env.n_actions, activation=tf.nn.softmax)(concat)
        value = Dense(1)(concat)
        return tf.keras.Model(inputs=[state, rnn_state], outputs=[action_prob, value, rnn_final_state])

policy = PongPolicy()

We will optimize the policy using the Asynchronous Advantage Actor Critic (A3C) algorithm.  There are lots of hyperparameters we could specify at this point, but the default values for most of them work well on this problem.  The only one we need to customize is the learning rate.

In [3]:
from deepchem.models.optimizers import Adam
a3c = dc.rl.A3C(env, policy, model_dir='model', optimizer=Adam(learning_rate=0.0002))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Optimize for as long as you have patience to.  By 1 million steps you should see clear signs of learning.  Around 3 million steps it should start to occasionally beat the game's built in AI.  By 7 million steps it should be winning almost every time.  Running on my laptop, training takes about 20 minutes for every million steps.

In [4]:
# Change this to train as many steps as you have patience for.
a3c.fit(1000)

Let's watch it play and see how it does! 

In [6]:
env.reset()
while not env.terminated:
    env.env.render()
    env.step(a3c.select_action(env.state))